## Recommendation system 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

/Users/opheliaming/opt/anaconda3/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/opheliaming/opt/anaconda3/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/opheliaming/opt/anaconda3/envs/py37/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
data = pd.read_json('song.json')
data['time_played'] = pd.to_datetime(data['time_played'])
data['user_sign_up_date'] = pd.to_datetime(data['user_sign_up_date'])
data.head()

,id,user_id,user_state,user_sign_up_date,song_played,time_played
0,GOQMMKSQQH,122,Louisiana,2015-05-16,Hey Jude,2015-06-11 21:51:35
1,HWKKBQKNWI,3,Ohio,2015-05-01,We Can Work It Out,2015-06-06 16:49:19
2,DKQSXVNJDH,35,New Jersey,2015-05-04,Back In the U.S.S.R.,2015-06-14 02:11:29
3,HLHRIDQTUW,126,Illinois,2015-05-16,P.s. I Love You,2015-06-08 12:26:10
4,SUKJCSBCYW,6,New Jersey,2015-05-01,Sgt. Pepper's Lonely Hearts Club Band,2015-06-28 14:57:00


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 6 columns):
id                   4000 non-null object
user_id              4000 non-null int64
user_state           4000 non-null object
user_sign_up_date    4000 non-null datetime64[ns]
song_played          4000 non-null object
time_played          4000 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 187.6+ KB


In [4]:
# check missing values
data.isnull().sum()

id                   0
user_id              0
user_state           0
user_sign_up_date    0
song_played          0
time_played          0
dtype: int64

In [5]:
# check unique values
for name in data.columns:
    print('{0:15s} \t {1:5d}'.format(name, len(data[name].unique())))

id              	  4000
user_id         	   196
user_state      	    41
user_sign_up_date 	    20
song_played     	   100
time_played     	  3997


In [36]:
state_user_count = data.groupby('user_state')['user_id'].apply(lambda x: len(np.unique(x))).reset_index()
state_user_count = state_user_count.rename(columns={'user_id': 'user_count'})
state_user_count = state_user_count.sort_values(by='user_count',ascending=False)

In [37]:
state_user_count

,user_state,user_count
25,New York,23
4,California,21
35,Texas,15
31,Pennsylvania,9
28,Ohio,9
10,Illinois,7
7,Florida,7
26,North Carolina,6
23,New Jersey,6
17,Massachusetts,6


In [38]:
state_user_count.head(3)

,user_state,user_count
25,New York,23
4,California,21
35,Texas,15


In [39]:
state_user_count.tail(9)

,user_state,user_count
24,New Mexico,1
9,Idaho,1
27,North Dakota,1
6,Connecticut,1
12,Iowa,1
32,Rhode Island,1
22,Nebraska,1
2,Arizona,1
13,Kansas,1


User egagement: measured through average plays per user in the given state: 

Number of total plays/Number of users

In [40]:
state_play_count = data.groupby('user_state')['id'].count().reset_index().rename(columns={'id': 'play_count'})
state_play_count 

,user_state,play_count
0,Alabama,104
1,Alaska,58
2,Arizona,22
3,Arkansas,34
4,California,425
5,Colorado,54
6,Connecticut,16
7,Florida,180
8,Georgia,135
9,Idaho,26


In [41]:
# merge user count and play count
state_user_play = pd.merge(state_user_count, state_play_count, on='user_state')
# calculate average play
state_user_play['average_play'] = state_user_play['play_count'] / state_user_play['user_count']
# sort according to average_play
state_user_play = state_user_play.sort_values(by='average_play', ascending=False)

In [42]:
state_user_play.head(3)

,user_state,user_count,play_count,average_play
38,Nebraska,1,36,36.000000
27,Alaska,2,58,29.000000
22,South Carolina,3,85,28.333333


In [43]:
# get the bottom 3 states
state_user_play.tail(3)

,user_state,user_count,play_count,average_play
17,Minnesota,4,42,10.5
30,Virginia,2,17,8.5
40,Kansas,1,8,8.0


In [44]:
def find_first_user(df):
    """ function to find the first user """
    idx = df['user_sign_up_date'].argmin()
    return df.loc[idx, ['user_id', 'user_sign_up_date']]

In [45]:
data.groupby('user_state').apply(find_first_user).sort_values(by='user_sign_up_date')

/Users/opheliaming/opt/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: 
The current behaviour of 'Series.argmin' is deprecated, use 'idxmin'
instead.
The behavior of 'argmin' will be corrected to return the positional
minimum in the future. For now, use 'series.values.argmin' or
'np.argmin(np.array(values))' to get the position of the minimum
row.
  This is separate from the ipykernel package so we can avoid doing imports until


,user_id,user_sign_up_date
user_state,,
Alabama,5,2015-05-01
Texas,7,2015-05-01
Oregon,1,2015-05-01
Ohio,3,2015-05-01
North Carolina,2,2015-05-01
New Mexico,4,2015-05-01
New Jersey,6,2015-05-01
Pennsylvania,11,2015-05-02
New York,19,2015-05-02


1. Build a function that takes as an input any of the songs in the data and returns the most likely song to be listened next. That is, if, for instance, a user is currently listening to "Eight Days A Week", which song has the highest probability of being played right after it by the same user? This is going to be v1 of a song recommendation model.
2. How would you set up a test to check whether your model works well and is improving engagement?

Different methods to answer this question:
1. The simplest method is to choose the most popular songs Or User's favoriate songs. 
3. Content-based filtering.
4. Collaborative filtering method.

## Collaborative filtering method

'Collaborative Filtering' idea, that is: if two songs are both liked by the same set of users, then they must be similar.

In [53]:
# Step 1: build the Song-User matrix
song_user = data.groupby(['song_played', 'user_id'])['id'].count().unstack(fill_value=0)
song_user.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,191,192,193,194,195,196,197,198,199,200
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,0,0,1,3,0,2,0,0,0,0,...,0,0,3,3,0,2,0,0,2,0
A Hard Day's Night,0,0,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,0
A Saturday Club Xmas/Crimble Medley,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ANYTIME AT ALL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Across The Universe,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
# Step 2: build song-song similarity matrix: 
#the similarity of two songs is calculate using the number of users whole listen them together.
# normalize each song's vector to have unit norm
# which simplifies 'cosine similarity' to dot-product of two vectors
song_user_norm = normalize(song_user, axis=1)  # normalize the song-user matrix
similarity = np.dot(song_user_norm, song_user_norm.T)  # calculate the similarity matrix
similarity_df = pd.DataFrame(similarity, index=song_user.index, columns=song_user.index)

similarity_df.head()

song_played,A Day In The Life,A Hard Day's Night,A Saturday Club Xmas/Crimble Medley,ANYTIME AT ALL,Across The Universe,All My Loving,All You Need Is Love,And Your Bird Can Sing,BAD BOY,BALLAD OF JOHN AND YOKO,...,We Can Work It Out,When I'm 64,While My Guitar Gently Weeps,Wild Honey Pie,With a Little Help From My Friends,YOUR MOTHER SHOULD KNOW,Yellow Submarine,Yesterday,You Never Give Me Your Money,You're Going To Lose That Girl
song_played,,,,,,,,,,,,,,,,,,,,,
A Day In The Life,1.000000,0.235702,0.074536,0.119523,0.212132,0.355023,0.329404,0.152145,0.210819,0.172133,...,0.464938,0.030429,0.508964,0.223607,0.359092,0.037268,0.318198,0.353220,0.087841,0.0
A Hard Day's Night,0.235702,1.000000,0.000000,0.000000,0.100000,0.136931,0.111803,0.000000,0.000000,0.091287,...,0.259548,0.129099,0.210099,0.000000,0.000000,0.000000,0.050000,0.195468,0.074536,0.0
A Saturday Club Xmas/Crimble Medley,0.074536,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.109435,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
ANYTIME AT ALL,0.119523,0.000000,0.000000,1.000000,0.000000,0.154303,0.094491,0.109109,0.000000,0.000000,...,0.116991,0.000000,0.138107,0.089087,0.183942,0.000000,0.000000,0.146845,0.000000,0.0
Across The Universe,0.212132,0.100000,0.000000,0.000000,1.000000,0.091287,0.000000,0.000000,0.000000,0.000000,...,0.138426,0.000000,0.116722,0.000000,0.000000,0.000000,0.000000,0.043437,0.000000,0.0


In [58]:
# Step 3: find the top-k most similar songs
def find_topk(song, similarity, k=1):
    df = similarity.loc[song].sort_values(ascending=False)[1:k + 1].reset_index()
    df = df.rename(columns={'song_played': 'Song', song: 'Similarity'})
    return df

In [59]:
# Example: find the top 10 similar song for 'A Day In The Life'
df = find_topk(song='A Day In The Life', similarity=similarity_df, k=10)
df

,Song,Similarity
0,Come Together,0.577357
1,Revolution,0.575179
2,Get Back,0.532342
3,While My Guitar Gently Weeps,0.508964
4,Back In the U.S.S.R.,0.492563
5,Hey Jude,0.490362
6,Let It Be,0.481286
7,Here Comes The Sun,0.475850
8,Lucy In The Sky With Diamonds,0.470427
9,Hello Goodbye,0.466242


## A/B test: How would you set up a test to check whether your model works well and is improving engagement?

1. Randomly split users into two groups, one Control group and one Experiment group. Where Control group has no recommendation strategy (random generate a song)and Experiment group recommend the next song.
2. After running some time, perform a one-tailed t-test on 'average #play per hour'
3. $H_0$: population 'average #play per hour' is same in two groups. $H_a$: experiment group's population 'average #play per hour' is higher than control group's